In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [2]:
'''
1. LH 단지정보
지역본부명, 세대수  -> '대구경북지역본부' 지역 쪼개기
지역본부명-> 지역으로 통일
공급유형 -> '국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'으로 분류

-새로운 데이터셋1 -
컬럼 
지역, 공급유형, 총세대수, 단지명, 주소 
# 인천이 어디에 속하는지 알 필요가 있음


2. train 
새로운 데이터셋1과 지역, 총세대수, 공급유형  공통값을 필터링

-새로운 데이터셋2 -
컬럼
단지코드, 지역, 공급유형, 총세대수, 단지명, 주소


3. 단지별 연령대별 성별정보
주택유형 필터['아파트']-> 데이터프레임1 생성
데이터프레임1에 도로명주소 str 필터 -> 지역 컬럼구분
지역, 세대수를 공통필터 -> 세대수구성 정보 획득


-새로운 데이터셋3 -
컬럼
단지코드, 지역, 공급유형, 총세대수, 단지명, 주소, 세대수구성원


4. 다시 트레인에 병합
단지코드를 기준으로 세대수구성원 병합
'''

"\n1. LH 단지정보\n지역본부명, 세대수  -> '대구경북지역본부' 지역 쪼개기\n지역본부명-> 지역으로 통일\n공급유형 -> '국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',\n       '장기전세', '공공분양', '행복주택', '공공임대(5년)'으로 분류\n\n-새로운 데이터셋1 -\n컬럼 \n지역, 공급유형, 총세대수, 단지명, 주소 \n# 인천이 어디에 속하는지 알 필요가 있음\n\n\n2. train \n새로운 데이터셋1과 지역, 총세대수, 공급유형  공통값을 필터링\n\n-새로운 데이터셋2 -\n컬럼\n단지코드, 지역, 공급유형, 총세대수, 단지명, 주소\n\n\n3. 단지별 연령대별 성별정보\n주택유형 필터['아파트']-> 데이터프레임1 생성\n데이터프레임1에 도로명주소 str 필터 -> 지역 컬럼구분\n지역, 세대수를 공통필터 -> 세대수구성 정보 획득\n\n\n-새로운 데이터셋3 -\n컬럼\n단지코드, 지역, 공급유형, 총세대수, 단지명, 주소, 세대수구성원\n\n\n4. 다시 트레인에 병합\n단지코드를 기준으로 세대수구성원 병합\n"

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
danji = pd.read_csv('공공데이터/한국토지주택공사_전국 LH아파트 단지정보_20210430.csv', encoding='cp949')
gender = pd.read_csv('공공데이터/한국토지주택공사_임대주택 단지별 연령대별 성별정보_20210511.csv', encoding='cp949')
display(train)
display(test)
display(danji)
display(gender)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


,지역본부명,단지코드,단지명,공급유형,주소,세대수,동수
0,서울지역본부,C02012,(NHF제4호)하남미사A4,공공임대(10년),"경기도 하남시 미사강변북로 25(선동) 경기 하남시 망월동, 풍산동, 선동, 덕풍동...",996,10
1,서울지역본부,C00458,가평읍내,공공임대(5년),NaN,355,5
2,서울지역본부,C00037,가평읍내,공공임대(50년),경기도 가평군 가평읍 가화로 142-26(가평읍내주공아파트),355,5
3,서울지역본부,C01592,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
4,서울지역본부,889,강남4BL,공공분양+토지임대부분양,서울특별시 강남구 자곡로3길 45(자곡동618),402,0
...,...,...,...,...,...,...,...
3658,세종특별본부,C02151,행정중심복합도시 3-1생활권 M5블록,공공임대(10년),세종특별자치시 대평로 56(대평동),1438,18
3659,세종특별본부,C02218,행정중심복합도시 3-2생활권 M2블록,국민임대,"세종특별자치시 보람1길 12(보람동,호려울마을2단지)",674,5
3660,세종특별본부,C02217,행정중심복합도시 3-3(M6)BL,공공분양,"세종특별자치시 남세종로 302(소담동,새샘마을6단지)",0,19
3661,세종특별본부,C02550,행정중심복합도시 3-3생활권 M2블록,국민임대,NaN,1100,4


,단지_일련번호,임대주택유형,공급기관명,주택명,주택유형,도로명주소,총세대수,10대미만(여자),10대미만(남자),10대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,30485277,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 19,9.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,30485278,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-2,8.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30485279,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-4,7.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,30485280,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로52번길 9-11,10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30485281,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로74번길 19,8.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19450,30855821,매입임대,NaN,매입다가구(전북김제시),다가구주택,NaN,0.0,1,4,2,...,0,0,1,0,0,0,0,0,0,0
19451,30855838,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,0,1,2,...,0,0,0,1,0,1,1,0,0,0
19452,30855839,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,1,3,2,...,2,2,0,0,1,1,0,0,0,0
19453,31284963,건설임대,NaN,양주옥정 A-20(2)(공임리츠),아파트,NaN,619.0,99,114,134,...,30,36,9,9,12,0,0,0,0,0


In [4]:
train['단지코드'].describe() # 423개 코드

count      2952
unique      423
top       C1439
freq         48
Name: 단지코드, dtype: object

In [5]:
test['단지코드'].describe() # 150개 코드

count      1022
unique      150
top       C2676
freq         46
Name: 단지코드, dtype: object

In [6]:
danji['지역본부명'].unique()

array(['서울지역본부', '강원지역본부', '경기지역본부', '인천지역본부', '충북지역본부', '대전충남지역본부',
       '전북지역본부', '광주전남지역본부', '대구경북지역본부', '경남지역본부', '부산울산지역본부', '제주지역본부',
       '세종특별본부'], dtype=object)

In [7]:
train['지역'].unique()

array(['경상북도', '경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도',
       '부산광역시', '제주특별자치도', '울산광역시', '충청북도', '전라남도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [8]:
a = test['지역'].unique()
display(a)

array(['경기도', '부산광역시', '전라북도', '경상남도', '충청남도', '대전광역시', '제주특별자치도', '강원도',
       '울산광역시', '경상북도', '충청북도', '광주광역시', '전라남도', '대구광역시', '세종특별자치시'],
      dtype=object)

In [9]:
train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [10]:
test['공급유형'].unique()

array(['국민임대', '영구임대', '임대상가', '공공임대(50년)', '공공임대(10년)', '공공임대(분납)',
       '행복주택'], dtype=object)

In [11]:
danji['공급유형'].unique()

array(['공공임대(10년)', '공공임대(5년)', '공공임대(50년)', '국민임대', '공공분양+토지임대부분양',
       '공공분양', '분양주택', '국민임대+영구임대', '행복주택', '공공분양+공공임대(5년)', '행복주택+임대상가',
       nan, '공공임대(분납)', '영구임대+임대상가', '공공임대(10년)+공공임대(분납)',
       '장기전세+국민임대+영구임대', '공공분양+공공임대(50년)', '영구임대', '외인임대',
       '공공분양+공공임대(10년)', '후분양', '토지임대부분양', '임대상가', '공공임대(10년)+임대상가',
       '공공임대(50년)+영구임대', '국민임대+행복주택'], dtype=object)

In [12]:
# 트레인과 테스트에 공급유형 결측이 없으므로, 결측이있는 단지데이터는 쓰지 않았다고 가정한다.
danji.isna().sum()

지역본부명      0
단지코드       0
단지명        1
공급유형     142
주소       773
세대수        0
동수         0
dtype: int64

In [13]:
# 결측치 대부분 입주자 모집중인 아파트
danji.isna().sum()
danji1 = danji.dropna(subset=['주소'])

In [14]:
경기 = danji1[danji1['주소'].str.contains('경기')]
경기['지역본부명'] = '경기도'

서울 = danji1[danji1['주소'].str.contains('서울')]
서울['지역본부명'] = '서울특별시'

충남 = danji1[danji1['주소'].str.contains('충청남도|충남')]
충남['지역본부명'] = '충청남도'

충북 = danji1[danji1['주소'].str.contains('충청북도|충북')]
충북['지역본부명'] = '충청북도'

대전 = danji1[danji1['주소'].str.contains('대전')]
대전['지역본부명'] = '대전광역시'

광주 = danji1[danji1['주소'].str.contains('광주')]
광주['지역본부명'] = '광주광역시'

전남 = danji1[danji1['주소'].str.contains('전라남도|전남')]
전남['지역본부명'] = '전라남도'

전북 = danji1[danji1['주소'].str.contains('전라북도|전북')]
전북['지역본부명'] = '전라북도'

부산 = danji1[danji1['주소'].str.contains('부산')]
부산['지역본부명'] = '부산광역시'

울산 = danji1[danji1['주소'].str.contains('울산')]
울산['지역본부명'] = '울산광역시'

대구 = danji1[danji1['주소'].str.contains('대구')]
대구['지역본부명'] = '대구광역시'

경북 = danji1[danji1['주소'].str.contains('경상북도|경북')]
경북['지역본부명'] = '경상북도'

경남 = danji1[danji1['주소'].str.contains('경상남도|경남')]
경남['지역본부명'] = '경상남도'

강원 = danji1[danji1['주소'].str.contains('강원도|강원')]
강원['지역본부명'] = '강원도'


제주 = danji1[danji1['주소'].str.contains('제주')]
제주['지역본부명'] = '제주특별자치도'

세종 = danji1[danji1['주소'].str.contains('세종')]
세종['지역본부명'] = '세종특별자치시'

temp1 = pd.concat([경기, 서울, 충남, 충북, 대전, 광주, 전남, 전북, 부산, 울산, 대구, 경북, 경남, 강원, 대구, 제주, 세종])
temp1.drop(['단지코드'], axis=1, inplace=True)
temp1.rename(columns={'지역본부명':'지역', '세대수':'총세대수'}, inplace=True)
temp1.reset_index(drop = True, inplace=True)
temp1

<ipython-input-14-2d43c2d35377>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경기['지역본부명'] = '경기도'
<ipython-input-14-2d43c2d35377>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  서울['지역본부명'] = '서울특별시'
<ipython-input-14-2d43c2d35377>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

,지역,단지명,공급유형,주소,총세대수,동수
0,경기도,(NHF제4호)하남미사A4,공공임대(10년),"경기도 하남시 미사강변북로 25(선동) 경기 하남시 망월동, 풍산동, 선동, 덕풍동...",996,10
1,경기도,가평읍내,공공임대(50년),경기도 가평군 가평읍 가화로 142-26(가평읍내주공아파트),355,5
2,경기도,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
3,경기도,고양 햇빛마을(중형임대),공공임대(5년),경기 고양시 덕양구 행신동 958(21/12) 햇빛마을,1,1
4,경기도,고양관산1,공공분양,경기 고양시 덕양구 관산동 996번지 주공그린빌,0,16
...,...,...,...,...,...,...
2843,세종특별자치시,행정중심복합도시 2-2(M8)BL,국민임대,"세종특별자치시 새롬중앙로 20(새롬동,새뜸마을8단지)",906,5
2844,세종특별자치시,행정중심복합도시 3-1생활권 M5블록,공공임대(10년),세종특별자치시 대평로 56(대평동),1438,18
2845,세종특별자치시,행정중심복합도시 3-2생활권 M2블록,국민임대,"세종특별자치시 보람1길 12(보람동,호려울마을2단지)",674,5
2846,세종특별자치시,행정중심복합도시 3-3(M6)BL,공공분양,"세종특별자치시 남세종로 302(소담동,새샘마을6단지)",0,19


In [15]:
국민 = temp1[temp1['공급유형'] == '국민임대']
국민['공급유형'] = '국민임대'

상가 = temp1[temp1['공급유형'] == '임대상가']
상가['공급유형'] = '임대상가'

영구 = temp1[temp1['공급유형'] == '영구임대']
영구['공급유형'] = '영구임대'

분납 = temp1[temp1['공급유형'] == '공공임대(분납)']
분납['공급유형'] = '공공임대(분납)'

오 = temp1[temp1['공급유형'] == '공공임대(5년)']
오['공급유형'] = '공공임대(5년)'

십 = temp1[temp1['공급유형'] == '공공임대(10년)']
십['공급유형'] = '공공임대(10년)'

오십 = temp1[temp1['공급유형'] == '공공임대(50년)']
오십['공급유형'] = '공공임대(50년)'

분양 = temp1[temp1['공급유형'] == '공공분양']
분양['공급유형'] = '공공분양'

전세 = temp1[temp1['공급유형'].str.contains('장기전세', na=False)]
전세['공급유형'] = '장기전세'

행복 = temp1[temp1['공급유형'] == '행복주택']
행복['공급유형'] = '행복주택'


temp2 = pd.concat([국민, 상가, 영구, 분납, 오, 십, 오십, 분양, 전세, 행복])
temp2.reset_index(drop = True, inplace=True)
temp2

<ipython-input-15-5b8085a22e87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  국민['공급유형'] = '국민임대'
<ipython-input-15-5b8085a22e87>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  상가['공급유형'] = '임대상가'
<ipython-input-15-5b8085a22e87>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,지역,단지명,공급유형,주소,총세대수,동수
0,경기도,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
1,경기도,남양주가운1,국민임대,"경기도 남양주시 가운로2길 28(다산동,남양주가운마을휴먼시아1단지)",1417,16
2,경기도,남양주가운2,국민임대,"경기도 남양주시 가운로2길 8(다산동,남양주가운마을휴먼시아2단지)",388,4
3,경기도,남양주마석,국민임대,경기도 남양주시 화도읍 맷돌로91번길 7,488,5
4,경기도,남양주미리내마을4-2단지,국민임대,"경기도 남양주시 별내4로 25(별내동,미리내마을 4-2단지)",1058,7
...,...,...,...,...,...,...
2178,강원도,춘천거두2 행복주택,행복주택,강원도 춘천시 동내면 거두택지길 70(춘천거두2지구행복주택) 춘천거두2 행복주택,480,3
2179,대구광역시,대구테크노폴리스 LH천년나무3단지,행복주택,대구광역시 달성군 현풍면 테크노북로4길 27(대구테크노폴리스엘에이치천년나무3단지),1020,4
2180,대구광역시,대구혁신 10단지(행복),행복주택,대구광역시 동구 메디밸리로 18(사복동) 대구신서혁신 LH천년나무10단지,1088,6
2181,제주특별자치도,서귀포서홍,행복주택,제주특별자치도 서귀포시 장수로 115(서홍동) 서귀포서홍행복주택,90,1


In [16]:
merge_df1 = pd.merge(train, temp2, on=['지역', '공급유형', '총세대수'], how='left')
merge_df1 = merge_df1[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소']]
display(merge_df1)

merge_df2 = pd.merge(test, temp2, on=['지역', '공급유형', '총세대수'], how='left')
merge_df2 = merge_df2[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소']]
display(merge_df2)
# merge_df1.to_csv('merge체크/LH 단지정보+트레인_merge체크_left.csv', encoding='cp949')
# merge_df2.to_csv('merge체크/LH 단지정보+테스트_merge체크_left.csv', encoding='cp949')

,단지코드,지역,공급유형,단지명,총세대수,주소
0,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
1,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
2,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
3,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
4,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)"
...,...,...,...,...,...,...
3202,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트)
3203,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트)
3204,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트)
3205,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트)


,단지코드,지역,공급유형,단지명,총세대수,주소
0,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
1,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
2,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
3,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
4,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)"
...,...,...,...,...,...,...
1108,C1267,경상남도,행복주택,NaN,675,NaN
1109,C2189,전라북도,국민임대,NaN,382,NaN
1110,C2189,전라북도,국민임대,NaN,382,NaN
1111,C2189,전라북도,국민임대,NaN,382,NaN


In [17]:
merge_df1['단지코드'].describe() # train 423개 코드

count      3207
unique      423
top       C1439
freq         48
Name: 단지코드, dtype: object

In [18]:
merge_df2['단지코드'].describe() #test 150개 코드

count      1113
unique      150
top       C2676
freq         46
Name: 단지코드, dtype: object

In [19]:
# merge_df1.to_csv('LH 단지정보+트레인.csv', encoding='cp949')
# merge_df2.to_csv('LH 단지정보+테스트.csv', encoding='cp949')

In [20]:
gender_apt = gender[gender['주택유형']=='아파트']
gender_apt.insert(2, '지역', '-1')

경기 = gender_apt[gender_apt['도로명주소'].str.contains('경기', na=False)]
경기['지역'] = '경기도'

서울 = gender_apt[gender_apt['도로명주소'].str.contains('서울', na=False)]
서울['지역'] = '서울특별시'

충남 = gender_apt[gender_apt['도로명주소'].str.contains('충청남도|충남', na=False)]
충남['지역'] = '충청남도'

충북 = gender_apt[gender_apt['도로명주소'].str.contains('충청북도|충북', na=False)]
충북['지역'] = '충청북도'

대전 = gender_apt[gender_apt['도로명주소'].str.contains('대전', na=False)]
대전['지역'] = '대전광역시'

광주 = gender_apt[gender_apt['도로명주소'].str.contains('광주', na=False)]
광주['지역'] = '광주광역시'

전남 = gender_apt[gender_apt['도로명주소'].str.contains('전라남도|전남', na=False)]
전남['지역'] = '전라남도'

전북 = gender_apt[gender_apt['도로명주소'].str.contains('전라북도|전북', na=False)]
전북['지역'] = '전라북도'

부산 = gender_apt[gender_apt['도로명주소'].str.contains('부산', na=False)]
부산['지역'] = '부산광역시'

울산 = gender_apt[gender_apt['도로명주소'].str.contains('울산', na=False)]
울산['지역'] = '울산광역시'

대구 = gender_apt[gender_apt['도로명주소'].str.contains('대구', na=False)]
대구['지역'] = '대구광역시'

경북 = gender_apt[gender_apt['도로명주소'].str.contains('경상북도|경북', na=False)]
경북['지역'] = '경상북도'

경남 = gender_apt[gender_apt['도로명주소'].str.contains('경상남도|경남', na=False)]
경남['지역'] = '경상남도'

강원 = gender_apt[gender_apt['도로명주소'].str.contains('강원도|강원', na=False)]
강원['지역'] = '강원도'

대구 = gender_apt[gender_apt['도로명주소'].str.contains('대구', na=False)]
대구['지역'] = '대구광역시'

제주 = gender_apt[gender_apt['도로명주소'].str.contains('제주', na=False)]
제주['지역'] = '제주특별자치도'

세종 = gender_apt[gender_apt['도로명주소'].str.contains('세종', na=False)]
세종['지역'] = '세종특별자치시'

temp3 = pd.concat([경기, 서울, 충남, 충북, 대전, 광주, 전남, 전북, 부산, 울산, 대구, 경북, 경남, 강원, 대구, 제주, 세종])
temp3

<ipython-input-20-93b65d611efc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  경기['지역'] = '경기도'
<ipython-input-20-93b65d611efc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  서울['지역'] = '서울특별시'
<ipython-input-20-93b65d611efc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,단지_일련번호,임대주택유형,지역,공급기관명,주택명,주택유형,도로명주소,총세대수,10대미만(여자),10대미만(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
21,4181,건설임대,경기도,성남시,선경단대영구임대아파트,아파트,경기도 성남시 수정구 산성대로383번길 28,60.0,0,0,...,17,8,6,5,4,6,3,0,0,0
188,30582333,건설임대,경기도,SH공사,수락리버시티1단지,아파트,경기도 의정부시 누원로 51,680.0,21,11,...,109,90,65,46,19,15,14,5,0,0
189,30582334,건설임대,경기도,SH공사,수락리버시티2단지,아파트,경기도 의정부시 누원로 52,473.0,13,9,...,27,36,16,15,12,5,7,2,1,1
2795,88058,매입임대,경기도,경기도시공사,NaN,아파트,경기도 광명시 목감로 58,112.0,9,10,...,21,10,2,7,3,0,1,0,0,0
2800,30819467,건설임대,경기도,경기도시공사,호수마을자연앤이편한세상,아파트,경기도 김포시 김포한강4로420번길 164,559.0,53,60,...,65,54,29,27,8,4,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,30701530,매입임대(단지유형),세종특별자치시,LH세종,세종영조정보마을,아파트,세종특별자치시 장군면 대학길 118,5.0,0,0,...,0,0,0,0,0,0,0,0,0,0
19352,30706817,매입임대(단지유형),세종특별자치시,LH세종,조치원 신동아파밀리에 3차아파트,아파트,세종특별자치시 조치원읍 도장말길 30,37.0,0,2,...,1,1,1,0,0,0,0,0,0,0
19353,30706818,매입임대(단지유형),세종특별자치시,LH세종,조치원 신동아파밀리에 3차아파트,아파트,세종특별자치시 조치원읍 도장말길 30,37.0,2,0,...,2,4,1,0,0,0,0,0,0,0
19354,30706820,매입임대(단지유형),세종특별자치시,LH세종,조치원 신동아파밀리에 3차아파트,아파트,세종특별자치시 조치원읍 도장말길 30,37.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
temp3.columns

Index(['단지_일련번호', '임대주택유형', '지역', '공급기관명', '주택명', '주택유형', '도로명주소', '총세대수',
       '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)'],
      dtype='object')

In [22]:
merge_df_res1 = pd.merge(merge_df1, temp3, on=['지역', '총세대수'], how='left')
merge_df_res1 = merge_df_res1[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소',  '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]
display(merge_df_res1)

merge_df_res2 = pd.merge(merge_df2, temp3, on=['지역', '총세대수'], how='left')
merge_df_res2 = merge_df_res2[['단지코드', '지역', '공급유형', '단지명', '총세대수', '주소',  '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]
display(merge_df_res2)

,단지코드,지역,공급유형,단지명,총세대수,주소,10대미만(여자),10대미만(남자),10대(여자),10대(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83.0,111.0,129.0,134.0,...,223.0,127.0,77.0,70.0,40.0,20.0,2.0,3.0,0.0,0.0
1,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83.0,111.0,129.0,134.0,...,223.0,127.0,77.0,70.0,40.0,20.0,2.0,3.0,0.0,0.0
2,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83.0,111.0,129.0,134.0,...,223.0,127.0,77.0,70.0,40.0,20.0,2.0,3.0,0.0,0.0
3,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83.0,111.0,129.0,134.0,...,223.0,127.0,77.0,70.0,40.0,20.0,2.0,3.0,0.0,0.0
4,C2483,경상북도,국민임대,포항장량 1단지,900,"경상북도 포항시 북구 장량중앙로 17(양덕동,장량 휴먼시아아파트)",83.0,111.0,129.0,134.0,...,223.0,127.0,77.0,70.0,40.0,20.0,2.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트),18.0,19.0,40.0,52.0,...,52.0,39.0,39.0,20.0,21.0,7.0,6.0,4.0,0.0,0.0
3831,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트),18.0,19.0,40.0,52.0,...,52.0,39.0,39.0,20.0,21.0,7.0,6.0,4.0,0.0,0.0
3832,C2532,강원도,국민임대,"평창하진부(07,매입) 1단지",239,강원도 평창군 진부면 경강로 3587(평창하진부주공아파트),18.0,19.0,40.0,52.0,...,52.0,39.0,39.0,20.0,21.0,7.0,6.0,4.0,0.0,0.0
3833,C2532,강원도,국민임대,평창하진부,239,강원도 평창군 진부면 경강로 3587 (평창하진부주공아파트),18.0,19.0,40.0,52.0,...,52.0,39.0,39.0,20.0,21.0,7.0,6.0,4.0,0.0,0.0


,단지코드,지역,공급유형,단지명,총세대수,주소,10대미만(여자),10대미만(남자),10대(여자),10대(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50.0,51.0,112.0,122.0,...,214.0,171.0,87.0,77.0,40.0,26.0,5.0,5.0,1.0,0.0
1,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50.0,51.0,112.0,122.0,...,214.0,171.0,87.0,77.0,40.0,26.0,5.0,5.0,1.0,0.0
2,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50.0,51.0,112.0,122.0,...,214.0,171.0,87.0,77.0,40.0,26.0,5.0,5.0,1.0,0.0
3,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50.0,51.0,112.0,122.0,...,214.0,171.0,87.0,77.0,40.0,26.0,5.0,5.0,1.0,0.0
4,C1072,경기도,국민임대,"광명역세권(03,택) 2단지",754,"경기도 광명시 서원로 25(소하동,광명역세권휴먼시아)",50.0,51.0,112.0,122.0,...,214.0,171.0,87.0,77.0,40.0,26.0,5.0,5.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388,C1267,경상남도,행복주택,NaN,675,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1389,C2189,전라북도,국민임대,NaN,382,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1390,C2189,전라북도,국민임대,NaN,382,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1391,C2189,전라북도,국민임대,NaN,382,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
merge_df_res1.drop_duplicates(['단지코드'], inplace=True)
merge_df_res2.drop_duplicates(['단지코드'], inplace=True)

In [24]:
display(merge_df_res1.loc[merge_df_res1.isnull()["10대미만(여자)"]])
display(merge_df_res2.loc[merge_df_res2.isnull()["10대미만(여자)"]])

,단지코드,지역,공급유형,단지명,총세대수,주소,10대미만(여자),10대미만(남자),10대(여자),10대(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
42,C1244,경기도,국민임대,성남판교 백현 3단지 (A25-1BL),1722,경기도 성남시 분당구 동판교로 92 (백현마을),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,C1925,강원도,영구임대,NaN,601,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,C1874,충청남도,영구임대,NaN,619,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,C1424,대전광역시,공공임대(50년),대전구봉8-2,625,"대전광역시 서구 관저로 84(관저동,구봉마을8단지)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,C2621,대전광역시,영구임대,NaN,1396,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,C2446,경기도,행복주택,의왕고천 A-1 행복주택리츠,2200,경기도 의왕시 행복마을3로 18(고천동),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3809,C2586,제주특별자치도,행복주택,서귀포서홍,90,제주특별자치도 서귀포시 장수로 115(서홍동) 서귀포서홍행복주택,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3814,C2035,강원도,국민임대,춘천우두 국민임대 3단지,492,강원도 춘천시 우두1길 40(우두동) 춘천우두 LH천년나무3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3819,C2020,부산광역시,행복주택,부산강서 행복주택,40,부산광역시 강서구 공항로1309번길 83-22(대저1동),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,단지코드,지역,공급유형,단지명,총세대수,주소,10대미만(여자),10대미만(남자),10대(여자),10대(남자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
77,C1006,대전광역시,영구임대,NaN,1505,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,C2676,부산광역시,영구임대,NaN,2572,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,C2177,충청남도,영구임대,NaN,1245,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,C2033,울산광역시,영구임대,NaN,1400,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,C1812,울산광역시,영구임대,NaN,1021,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,C2253,강원도,영구임대,NaN,1161,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,C1729,강원도,영구임대,NaN,627,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,C1392,경상북도,국민임대,안동송현휴먼시아,794,"경상북도 안동시 송현길 16(송현동,송현휴먼시아3단지)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,C2430,대전광역시,공공임대(10년),NaN,687,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1054,C1712,경상북도,국민임대,"상주무양(07,국민) 7단지",737,"경상북도 상주시 동수4길 55(무양동,상주무양LH7단지)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# for i in merge_df_res1.index:
#     if pd.notnull(merge_df_res1.ix[i,'단지명']) == True:
#         merge_df_res1[]
        

In [26]:
# merge_df_res1 = merge_df_res1[['단지코드', '단지명', '주소', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
#        '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
#        '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
#        '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

# merge_df_res2 = merge_df_res2[['단지코드', '단지명', '주소', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
#        '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
#        '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
#        '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

# display(merge_df_res1)
# display(merge_df_res2)

In [27]:
merge_df_res1 = merge_df_res1[['단지코드', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

merge_df_res2 = merge_df_res2[['단지코드', '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '20대(남자)',
       '30대(여자)', '30대(남자)', '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)',
       '60대(여자)', '60대(남자)', '70대(여자)', '70대(남자)', '80대(여자)', '80대(남자)',
       '90대(여자)', '90대(남자)', '100대(여자)', '100대(남자)']]

display(merge_df_res1)
display(merge_df_res2)

,단지코드,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2483,83.0,111.0,129.0,134.0,112.0,116.0,124.0,104.0,150.0,...,223.0,127.0,77.0,70.0,40.0,20.0,2.0,3.0,0.0,0.0
8,C2515,26.0,30.0,93.0,83.0,67.0,82.0,49.0,46.0,89.0,...,93.0,119.0,57.0,42.0,38.0,12.0,7.0,2.0,0.0,0.0
15,C1407,93.0,115.0,141.0,167.0,176.0,162.0,256.0,217.0,270.0,...,240.0,157.0,89.0,70.0,50.0,31.0,11.0,1.0,1.0,0.0
26,C1945,48.0,48.0,151.0,172.0,152.0,144.0,88.0,77.0,199.0,...,135.0,94.0,101.0,77.0,66.0,29.0,9.0,2.0,2.0,0.0
32,C1470,77.0,75.0,107.0,128.0,95.0,109.0,121.0,101.0,151.0,...,132.0,114.0,50.0,56.0,28.0,10.0,10.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,C2586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3814,C2035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3819,C2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3822,C2437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,단지코드,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,50.0,51.0,112.0,122.0,125.0,144.0,96.0,119.0,172.0,...,214.0,171.0,87.0,77.0,40.0,26.0,5.0,5.0,1.0,0.0
8,C1128,85.0,79.0,254.0,302.0,224.0,231.0,177.0,146.0,324.0,...,329.0,235.0,183.0,136.0,84.0,46.0,24.0,5.0,0.0,0.0
17,C1456,26.0,19.0,67.0,68.0,80.0,95.0,59.0,67.0,96.0,...,198.0,104.0,88.0,70.0,40.0,18.0,8.0,2.0,0.0,0.0
26,C1840,45.0,43.0,80.0,73.0,95.0,78.0,85.0,85.0,96.0,...,101.0,88.0,58.0,36.0,23.0,12.0,3.0,2.0,0.0,0.0
30,C1332,107.0,141.0,260.0,311.0,251.0,283.0,206.0,157.0,313.0,...,306.0,228.0,105.0,94.0,84.0,38.0,21.0,5.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,C2456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1371,C1266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1376,C2152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378,C1267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
merge_df_res1.to_csv('data/정제된 데이터/트레인_단지_세대원_재병합이전.csv', encoding='cp949')
merge_df_res2.to_csv('data/정제된 데이터/테스트_단지_세대원_재병합이전.csv', encoding='cp949')

In [29]:
# result_train = pd.merge(train, merge_df_res1, on=['단지코드'])
# result_test = pd.merge(test, merge_df_res2, on=['단지코드'])
# display(result_train)
# display(result_test)

In [30]:
# result_train.to_csv('data/합친데이터_트레인_최종20210625.csv')
# result_test.to_csv('data/합친데이터_테스트_최종20210625.csv')